# Building Datasets

In most of our examples, we use dataset_loader to avoid boilerplate code when training fair classifiers. 
This notebook sets out how to create similar code for new datasets.

For evaluating and fitting fair classifiers we require access to the group each datapoint is assigned to and the target (i.e. ground-truth) label the classifier is trying to predict. 

For sklearn, classifiers assume that they only recieve the data used to predict, and as the target labels should never be passed with the rest of the data, and the groups should only be passed if the classifier uses them directly (i.e. if we are not using infered attributes).



There are four important cases:
1. Fair Classifiers using autogluon.
    Create a dataframe or tabular dataset containing all data used for classification, target labels, and groups.
    Autogluon takes pandas dataframes or their own internal tabular dataset type and only uses the columns the model was trained on to classify the data.
    When using infered attributes you should ensure that neither the classifier predicting groups nor the classifier predicting target labels has access to the groups or target labels at training time. This is taken care for you automatically by using  `oxonfair.inferred_attribute_builder`.
2. Fair Classifiers using Sklearn with known attributes.
    Create a dataset by calling `oxonfair.build_data_dict` with two arguments - the target labels `y` and the data `X` used by the classifier. 
3. Fair Classifiers using Sklearn with inferred attributes. 
    Create a dataset by calling `oxonfair.build_data_dict` with three arguments - the target labels `y`, the data `X` used by the classifier, and the groups. 
4. Fair Classifiers using Deep networks.
    Create a classifier by calling `oxonfair.DeepFairPredictor` with three arguments - the target labels, the predictions made by the classifier, and the groups. See [this notebook for examples](quickstart_DeepFairPredictor_computer_vision.ipynb).

## AutoGluon Example


In [1]:
from autogluon.tabular import TabularDataset, TabularPredictor
import oxonfair
from oxonfair import FairPredictor, inferred_attribute_builder 
from oxonfair.utils import group_metrics as gm
train_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/train.csv')
test_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv')
# Train base classifier
predictor = TabularPredictor(label='class').fit(train_data=train_data, time_limit=3)

/opt/miniconda3/envs/ag/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


No path specified. Models will be saved in: "AutogluonModels/ag-20240617_142353"


No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.


Beginning AutoGluon training ... Time limit = 3s


AutoGluon will save models to "AutogluonModels/ag-20240617_142353"


...

	0.8672	 = Validation score   (accuracy)


	0.02s	 = Training   runtime


	0.0s	 = Validation runtime


AutoGluon training complete, total runtime = 4.74s ... Best model: "WeightedEnsemble_L2"


TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels/ag-20240617_142353")


In [2]:
# Modify predictor to enforce fairness over the train_data with respect to groups given by the column 'sex'
fpredictor = FairPredictor(predictor,train_data,'sex')
# Maximize accuracy while enforcing that the demographic parity (the difference in positive decision rates between men and women is at most 0.02)
fpredictor.fit(gm.accuracy,gm.demographic_parity,0.02)
#Evaluate per group on test data
fpredictor.evaluate_groups(test_data)

Accuracy  Balanced Accuracy  F1 score       MCC  \
         Groups                                                                
original Overall             0.863036           0.771720  0.674453  0.597347   
          Female             0.934631           0.760266  0.638655  0.617120   
          Male               0.826698           0.765123  0.680512  0.571345   
         Maximum difference  0.107933           0.004857  0.041857  0.045775   
updated  Overall             0.843177           0.736263  0.617191  0.532205   
          Female             0.894193           0.859737  0.623377  0.587947   
          Male               0.817284           0.722584  0.615335  0.542526   
         Maximum difference  0.076909           0.137153  0.008042  0.045421   

                             Precision    Recall   ROC AUC  Positive Count  \
         Groups                                                              
original Overall              0.773438  0.597929  0.918163          2318.0   
          Female              0.785124  0.538244  0.938701           353.0   
          Male                0.771613  0.608651  0.896625          1965.0   
         Maximum difference   0.013511  0.070408  0.042075          1612.0   
updated  Overall              0.733373  0.532787  0.819932          2318.0   
          Female              0.504378  0.815864  0.938701           353.0   
          Male                0.850854  0.481934  0.896625          1965.0   
         Maximum difference   0.346475  0.333930  0.042075          1612.0   

                             Negative Count  Positive Label Rate  \
         Groups                                                    
original Overall                     7451.0             0.237281   
          Female                     2936.0             0.107327   
          Male                       4515.0             0.303241   
         Maximum difference          1579.0             0.195913   
updated  Overall                     7451.0             0.237281   
          Female                     2936.0             0.107327   
          Male                       4515.0             0.303241   
         Maximum difference          1579.0             0.195913   

                             Positive Prediction Rate  
         Groups                                        
original Overall                             0.183437  
          Female                             0.073579  
          Male                               0.239198  
         Maximum difference                  0.165619  
updated  Overall                             0.172382  
          Female                             0.173609  
          Male                               0.171759  
         Maximum difference                  0.001850

# Autogluon with inferered attributes 

In [3]:
predictor2, protected = inferred_attribute_builder(train_data, 'class', 'sex', time_limit=3)
# Modify predictor to enforce fairness over the train_data with respect to groups given by the column 'sex'
fpredictor = FairPredictor(predictor2, train_data, 'sex', inferred_groups=protected)
# Maximize accuracy while enforcing that the demographic parity (the difference in positive decision rates between men and women is at most 0.02)
fpredictor.fit(gm.accuracy,gm.demographic_parity,0.02)
#Evaluate per group on test data
fpredictor.evaluate_groups(test_data)

No path specified. Models will be saved in: "AutogluonModels/ag-20240617_142358"


No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.


Beginning AutoGluon training ... Time limit = 3s


AutoGluon will save models to "AutogluonModels/ag-20240617_142358"


=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.10.13
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 23.5.0: Wed May  1 20:14:38 PDT 2024; root:xnu-10063.121.3~5/RELEASE_ARM64_T6020
CPU Count:          10
Memory Avail:       6.16 GB / 16.00 GB (38.5%)
Disk Space Avail:   363.53 GB / 460.43 GB (79.0%)


...

	0.03s	 = Training   runtime


	0.0s	 = Validation runtime


AutoGluon training complete, total runtime = 3.23s ... Best model: "WeightedEnsemble_L2"


TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels/ag-20240617_142401")


Accuracy  Balanced Accuracy  F1 score       MCC  \
         Groups                                                                
original Overall             0.866721           0.779485  0.685962  0.609715   
          Female             0.936151           0.771087  0.653465  0.629785   
          Male               0.831481           0.772580  0.691525  0.584217   
         Maximum difference  0.104669           0.001493  0.038060  0.045568   
updated  Overall             0.847681           0.737878  0.622335  0.543907   
          Female             0.906659           0.848028  0.640094  0.600067   
          Male               0.817747           0.723778  0.617428  0.543701   
         Maximum difference  0.088912           0.124249  0.022666  0.056366   

                             Precision    Recall   ROC AUC  Positive Count  \
         Groups                                                              
original Overall              0.777899  0.613460  0.918873          2318.0   
          Female              0.782609  0.560907  0.941070           353.0   
          Male                0.777143  0.622901  0.897262          1965.0   
         Maximum difference   0.005466  0.061994  0.043808          1612.0   
updated  Overall              0.755857  0.528904  0.834367          2318.0   
          Female              0.546000  0.773371  0.914831           353.0   
          Male                0.849376  0.484987  0.862494          1965.0   
         Maximum difference   0.303376  0.288384  0.052337          1612.0   

                             Negative Count  Positive Label Rate  \
         Groups                                                    
original Overall                     7451.0             0.237281   
          Female                     2936.0             0.107327   
          Male                       4515.0             0.303241   
         Maximum difference          1579.0             0.195913   
updated  Overall                     7451.0             0.237281   
          Female                     2936.0             0.107327   
          Male                       4515.0             0.303241   
         Maximum difference          1579.0             0.195913   

                             Positive Prediction Rate  
         Groups                                        
original Overall                             0.187123  
          Female                             0.076923  
          Male                               0.243056  
         Maximum difference                  0.166132  
updated  Overall                             0.166035  
          Female                             0.152022  
          Male                               0.173148  
         Maximum difference                  0.021126

## SKlearn with known attributes.

In [4]:
import xgboost
import pandas as pd
data=pd.concat((train_data, test_data))
target = data['class']!= ' <=50K'
data.drop('class',inplace=True,axis=1)
data = pd.get_dummies(data)
# get_dumies must be called on all data.
# Otherwise it breaks if either train and test contain a feature missing from the other.
training_data = data.sample(frac=0.7)
training_target = target.iloc[training_data.index]
testing_data = data.drop(training_data.index)
testing_target = target.iloc[testing_data.index]

### Unlike autogluon much of sklearn breaks if you pass round dataframes 
### containing features such as target labels, and groups that are not used by the classifier.
### We pass dictionaries that represent the entire dataset to get round this.
### They contain 'target' 'data', 'groups' (optional), and 'factor' (optional)
    
training_set = oxonfair.build_data_dict(training_target,training_data)
testing_set = oxonfair.build_data_dict(testing_target, testing_data) 
#train base classifier
classifier = xgboost.XGBClassifier()
classifier.fit(y=training_target, X=training_data)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [5]:
# Modify predictor to enforce fairness over the train_data with respect to groups given by the column 'sex'
fpredictor = FairPredictor(classifier, training_set, 'sex_ Female')
# Maximize accuracy while enforcing that the demographic parity (the difference in positive decision rates between men and women is at most 0.02)
fpredictor.fit(gm.accuracy, gm.demographic_parity,0.02)
#Evaluate per group on test data
fpredictor.evaluate_groups(testing_set)

Accuracy  Balanced Accuracy  F1 score       MCC  \
         Groups                                                                
original Overall             0.848938           0.742409  0.631163  0.553226   
         False               0.818143           0.743533  0.645656  0.537888   
         True                0.913544           0.701361  0.549923  0.541227   
         Maximum difference  0.095400           0.042172  0.095733  0.003340   
updated  Overall             0.836937           0.703423  0.568331  0.505734   
         False               0.804219           0.689733  0.552987  0.494518   
         True                0.905577           0.783503  0.624413  0.570435   
         Maximum difference  0.101357           0.093771  0.071427  0.075918   

                             Precision    Recall   ROC AUC  Positive Count  \
         Groups                                                              
original Overall              0.765369  0.537000  0.880708          2527.0   
         False                0.759510  0.561487  0.868309          2098.0   
         True                 0.806306  0.417249  0.863653           429.0   
         Maximum difference   0.046796  0.144238  0.004656          1669.0   
updated  Overall              0.783183  0.445983  0.797238          2527.0   
         False                0.847441  0.410391  0.868309          2098.0   
         True                 0.628842  0.620047  0.863653           429.0   
         Maximum difference   0.218599  0.209656  0.004656          1669.0   

                             Negative Count  Positive Label Rate  \
         Groups                                                    
original Overall                     7972.0             0.240690   
         False                       5012.0             0.295077   
         True                        2960.0             0.126586   
         Maximum difference          2052.0             0.168491   
updated  Overall                     7972.0             0.240690   
         False                       5012.0             0.295077   
         True                        2960.0             0.126586   
         Maximum difference          2052.0             0.168491   

                             Positive Prediction Rate  
         Groups                                        
original Overall                             0.168873  
         False                               0.218143  
         True                                0.065506  
         Maximum difference                  0.152637  
updated  Overall                             0.137061  
         False                               0.142897  
         True                                0.124816  
         Maximum difference                  0.018082

In [6]:
# We can also make predictions on data that doesn't have target labels attached.
fpredictor.predict(testing_data)

array([0, 0, 0, ..., 0, 0, 1])

## SKlearn with infered groups

In [7]:
y_train = training_set['target']
groups_train = training_set['data']['sex_ Female']
X_train = training_set['data'].drop('sex_ Female', axis=1)
training_set = oxonfair.build_data_dict(y_train, X_train,groups_train)
y_test = testing_set['target']
groups_test = testing_set['data']['sex_ Female']
X_test = testing_set['data'].drop('sex_ Female', axis=1)
test_set = oxonfair.build_data_dict(y_test, X_test,groups_test)

#train base classifiers
classifier = xgboost.XGBClassifier()
classifier.fit(y=y_train,X=X_train)
group_classifier = xgboost.XGBClassifier()
group_classifier.fit(y=groups_train,X=X_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [8]:
# Modify predictor to enforce fairness over the train_data with respect to groups given by the column 'sex'
fpredictor = FairPredictor(classifier, training_set, inferred_groups=group_classifier)
# Maximize accuracy while enforcing that the demographic parity (the difference in positive decision rates between men and women is at most 0.02)
fpredictor.fit(gm.accuracy,gm.demographic_parity,0.02)
#Evaluate per group on test data
fpredictor.evaluate_groups(test_set)

Some groups were not assigned, we only saw: [1 2]


Accuracy  Balanced Accuracy  F1 score       MCC  \
         Groups                                                                
original Overall             0.848938           0.742409  0.631163  0.553226   
         False               0.818143           0.743533  0.645656  0.537888   
         True                0.913544           0.701361  0.549923  0.541227   
         Maximum difference  0.095400           0.042172  0.095733  0.003340   
updated  Overall             0.836937           0.703423  0.568331  0.505734   
         False               0.804219           0.689733  0.552987  0.494518   
         True                0.905577           0.783503  0.624413  0.570435   
         Maximum difference  0.101357           0.093771  0.071427  0.075918   

                             Precision    Recall   ROC AUC  Positive Count  \
         Groups                                                              
original Overall              0.765369  0.537000  0.880708          2527.0   
         False                0.759510  0.561487  0.868309          2098.0   
         True                 0.806306  0.417249  0.863653           429.0   
         Maximum difference   0.046796  0.144238  0.004656          1669.0   
updated  Overall              0.783183  0.445983  0.797238          2527.0   
         False                0.847441  0.410391  0.868309          2098.0   
         True                 0.628842  0.620047  0.863653           429.0   
         Maximum difference   0.218599  0.209656  0.004656          1669.0   

                             Negative Count  Positive Label Rate  \
         Groups                                                    
original Overall                     7972.0             0.240690   
         False                       5012.0             0.295077   
         True                        2960.0             0.126586   
         Maximum difference          2052.0             0.168491   
updated  Overall                     7972.0             0.240690   
         False                       5012.0             0.295077   
         True                        2960.0             0.126586   
         Maximum difference          2052.0             0.168491   

                             Positive Prediction Rate  
         Groups                                        
original Overall                             0.168873  
         False                               0.218143  
         True                                0.065506  
         Maximum difference                  0.152637  
updated  Overall                             0.137061  
         False                               0.142897  
         True                                0.124816  
         Maximum difference                  0.018082

In [9]:
# We can also make predictions on data that doesn't have target labels attached.
fpredictor.predict(test_set['data'])

array([0, 0, 0, ..., 0, 0, 1])